## Historical Data

In [0]:
# Set the current database to 'kelvin'
spark.sql("USE hive_metastore.kelvin")

# Load the table 'pcp' into a DataFrame
df = spark.table("pcp_historical_data")

# Convert to pandas and clean 
df = df.toPandas()
df = df.dropna()
df.head()

,time,casing_pressure,tubing_pressure,water_flow_rate,gas_flow_rate,torque,speed
0,2023-12-13 06:00:00,56.021122,57.0,21.325994,41.126305,94.019989,54.899994
1,2023-12-13 06:00:30,56.017956,57.0,24.394835,40.480297,93.929993,55.000000
2,2023-12-13 06:01:00,56.034664,57.0,27.043316,40.693230,93.819977,54.799988
3,2023-12-13 06:01:30,56.033863,57.0,25.733318,40.977997,93.959991,55.000000
4,2023-12-13 06:02:00,56.049057,57.0,24.749817,40.067261,93.699982,55.000000


## Synthetic Data

In [0]:
import pandas as pd
import numpy as np

# Sample data creation
np.random.seed(0)

df = pd.DataFrame({
    'casing_pressure': np.random.normal(100, 10, 100),
    'tubing_pressure': np.random.normal(90, 10, 100),
    'gas_flow_rate': np.random.normal(50, 5, 100),
    'water_flow_rate': np.random.normal(20, 2, 100),
    'speed': np.random.normal(100, 20, 100)  # This is the target variable
})
df.head()

,casing_pressure,tubing_pressure,gas_flow_rate,water_flow_rate,speed
0,117.640523,108.831507,48.154091,17.386946,88.026921
1,104.001572,76.522409,48.803104,23.316261,77.682060
2,109.787380,77.295150,55.498298,19.763672,115.333264
3,122.408932,99.693967,53.276319,18.639644,107.125856
4,118.675580,78.268766,53.200658,21.332766,64.629231


## Train Model

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Data Preprocessing
scaler = StandardScaler()
features = ['casing_pressure', 'tubing_pressure', 'gas_flow_rate', 'water_flow_rate']

X = df[features]
y = df['speed']

# Scaling and converting back to DataFrame to keep column names
X_scaled_df = pd.DataFrame(scaler.fit_transform(X), columns=features)

# Splitting the Data chronologically
X_train = X_scaled_df.iloc[:int(X_scaled_df.shape[0]*0.8), :]
X_test = X_scaled_df.iloc[int(X_scaled_df.shape[0]*0.8):, :]

y_train = y.iloc[:int(y.shape[0]*0.8)]
y_test = y.iloc[int(y.shape[0]*0.8):]

# Model Selection and Training
model = LinearRegression()
model.fit(X_train, y_train)

Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

LinearRegression()

## Evaluate

In [0]:
# Sample data creation
np.random.seed(0)

eval_df = pd.DataFrame({
    'casing_pressure': np.random.normal(100, 10, 10),
    'tubing_pressure': np.random.normal(90, 10, 10),
    'gas_flow_rate': np.random.normal(50, 5, 10),
    'water_flow_rate': np.random.normal(20, 2, 10)
})

scaler = StandardScaler()
eval_df = pd.DataFrame(scaler.fit_transform(eval_df), columns=features)

# Run model prediction
predicted_speeds = model.predict(eval_df)
predicted_speeds

array([ 92.61976766,  97.01521079, 101.94529251, 101.76229705,
       103.25712154,  97.31157131,  92.76460528,  97.55602505,
       104.18130794, 106.07196261])

## Insert recommendations to Delta Table

In [0]:
recs_df = pd.DataFrame({
    'timestamp': [pd.Timestamp.now()] * len(predicted_speeds),
    'asset': ['pcp_' + str(i).zfill(2) for i in range(1, len(predicted_speeds) + 1)],
    'recommended_speed': np.round(predicted_speeds, 1)
})
recs_df.head()


,timestamp,asset,recommended_speed
0,2024-05-01 23:34:32.525690,pcp_01,92.6
1,2024-05-01 23:34:32.525690,pcp_02,97.0
2,2024-05-01 23:34:32.525690,pcp_03,101.9
3,2024-05-01 23:34:32.525690,pcp_04,101.8
4,2024-05-01 23:34:32.525690,pcp_05,103.3


In [0]:
# Convert df to a Spark DataFrame
result_df = spark.createDataFrame(recs_df)

# Write the Spark DataFrame to the Delta table
result_df.write.format("delta").mode("append").saveAsTable("kelvin.pcp_optimization_recommendations")

In [0]:
%sql
SELECT * FROM kelvin.pcp_optimization_recommendations ORDER BY `timestamp` DESC

timestamp,asset,recommended_speed
2024-05-01T23:34:32.52569Z,pcp_01,92.6
2024-05-01T23:34:32.52569Z,pcp_02,97.0
2024-05-01T23:34:32.52569Z,pcp_03,101.9
2024-05-01T23:34:32.52569Z,pcp_04,101.8
2024-05-01T23:34:32.52569Z,pcp_05,103.3
2024-05-01T23:34:32.52569Z,pcp_06,97.3
2024-05-01T23:34:32.52569Z,pcp_07,92.8
2024-05-01T23:34:32.52569Z,pcp_08,97.6
2024-05-01T23:34:32.52569Z,pcp_09,104.2
2024-05-01T23:34:32.52569Z,pcp_10,106.1
